In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
**Task Definition**
You are an **SMT expert** specializing in **Semantic Fusion**, a method for validating SMT solvers. Your task is to **automatically fuse two satisfiable SMT formulas** into a new formula
 while **preserving satisfiability.**

The formulas are written in **SMT-LIB syntax**. Your goal is to take two input formulas **ω1 and ω2**, perform **Semantic Fusion**, and return a **new formula ω12** that remains satisfiable by construction.

### SMT-LIB Syntax Guidelines:
**The generated output must strictly conform to the SMT-LIB format and include**:

1.	Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
2.	Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
3.	Assertions: Use assert to specify constraints: (assert <constraint>).
4.	Boolean Expressions:
	•	Logical AND: (and <expr1> <expr2> ...)
	•	Logical OR: (or <expr1> <expr2> ...)
	•	Logical NOT: (not <expr>)
5.	Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
6.	Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
7.	Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
8.	Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
9.	Quantifiers: Use forall and exists for quantified expressions:
	•	Universal: (forall ((x <type>) ...) <body>)
	•	Existential: (exists ((x <type>) ...) <body>)
10.	Constants: Declare constants directly using their types:
	•	Integers: 1, -3
	•	Real numbers: 1.0, -2.5
	•	Booleans: true, false
11.	Parentheses Matching: Ensure all expressions are enclosed in properly matched parentheses.
12.	Comments: Use ; for single-line comments in SMT-LIB.

**Avoid invalid SMT-LIB constructs! Ensure all parentheses are correctly matched.**

**Semantic Fusion Steps**:

1. Merge ω1 and ω2 into a single formula ω1 ∧ ω2, preserving original declarations and assertions.

2. Introduce a fresh variable z which is not present in either input formula.

3. Define the fusion function: z = x * y

4. Derive inversion functions:
	•	rx(y, z) = z div y  (expression for x)
	•	ry(x, z) = z div x  (expression for y)
    
    Derive expressions for `rx(y, z)` and `ry(x, z)`. Since `z = x * y`, we can express `x` as `z div y` and `y` as `z div x`.

5.	Substitute variables:
	•	Replace the leftmost occurrence of x in ω1 **with the explicit expression** (z div y), **NOT** rx(y, z).
	•	Replace the rightmost occurrence of y in ω2 **with the explicit expression** (z div x), **NOT** ry(x, z).

6. **DO NOT introduce new assertions**—the fused formula should be **syntactically valid SMT-LIB** without modifying logical constraints.

7. Do NOT add any new assert blocks to the conjoined formula!


###Example

**Formula1(ω1)**:
(declare-fun x () Int)
(assert (or (> x 0) (> x 1)))

**Formula2(ω2)**:
(declare-fun ε () Int)
(assert (or (< ε 0) (< ε 1)))


**Expected Output**

(declare-fun x () Int)
(declare-fun ε () Int)
(declare-fun z () Int)

; Fusion: Introduce z = x * ε
(assert (or 
    (or (> x 0) (> (z div ε) 1)) ; Replace first x with (z div ε)
    (or (< (z div x) 0) (< ε 1)) ; Replace first ε with (z div x)
))

###Additional Constraints
- Provide the result after performing each step.
- **Only one occurrence of x and ε should be replaced**.
- **No new assertions should be added**, only substitutions.
- **Variable names must remain unchanged**, except for z.
- Refrain from inserting comments in the final fused formula.

Now, perform the described operation for these formulas:

        """
        ),
        (
            "human",
            """
ω1: {formula1}
ω2: {formula2}
"""
    ),
]
)

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2
)

In [25]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # base_url="...",
    # organization="...",
    # other params...
)

In [3]:
chain = prompt | llm
formula1 = """
( declare-fun x () Int )
( declare-fun w () Bool )

( assert (= x (- 1) ) )
( assert (= w (= x (- 1) ) ) )
( assert w )
"""

formula2 = """
( declare-fun y () Int )
( declare-fun v () Bool )

( assert (= v ( not (= y (- 1) ) ) ) )
( assert ( ite v false (= y (- 1) ) ) )
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

Let's perform the Semantic Fusion step-by-step for the given formulas ω1 and ω2.

### Step 1: Merge ω1 and ω2

First, we combine the declarations and assertions from both formulas:

```smtlib
(declare-fun x () Int)
(declare-fun w () Bool)
(declare-fun y () Int)
(declare-fun v () Bool)

(assert (= x (- 1)))
(assert (= w (= x (- 1))))
(assert w)
(assert (= v (not (= y (- 1)))))
(assert (ite v false (= y (- 1))))
```

### Step 2: Introduce a fresh variable z

We introduce a new variable `z` of type `Int`:

```smtlib
(declare-fun z () Int)
```

### Step 3: Define the fusion function

We define the fusion function `z = x * y`. However, since we are not adding new assertions, we will not explicitly write this as an assertion. Instead, we will use it to derive the inversion functions.

### Step 4: Derive inversion functions

From `z = x * y`, we derive:
- `rx(y, z) = z div y`
- `ry(x, z) = z div x`

### Step 5: Substitute variables

We need to replace the leftmost occurrence of `x` in ω1 with